# Programming Assignment 6.2. Optimizing a reversible circuit

In this task, you'll optimize the given circuit - modify it so that its effect doesn't change (it still implements the same computation) and it requires as few resources to run as possible. See `evaluate_results` function for details on how your absolute score for the task is calculated.

In [ ]:
import qsharp

**The complete code for the task should be in this cell.**   
This cell can include additional open statements and helper operations and functions if your solution needs them. Your solution should not use measurements.

In [ ]:
// C -  3 - 00011
// O - 15 - 01111
// M - 13 - 01101 
// P - 16 - 10000 
// U - 21 - 10101
// T - 20 - 10100
// E -  5 - 00101
// R - 18 - 10010

// Convert to 

// '00011',
// '00101',
// '011x1',
// '100x0',
// '1010x'

%%qsharp
open Microsoft.Quantum.Canon;

// This circuit implements a marking oracle that marks the letters that comprise the word COMPUTER.
// The oracle uses little endian to convert integers that represent letters positions in the aphabet to bit strings.
// The input will contain exactly 5 qubits.
operation QuantumCircuit(x : Qubit[], target : Qubit) : Unit is Adj {
    ApplyControlledOnBitString( [false, false, false, true, true], X, [x[0], x[1], x[2], x[3], [4]], target);
    ApplyControlledOnBitString( [false, false, true, false, true], X, [x[0], x[1], x[2], x[3], [4]], target);
    ApplyControlledOnBitString( [false, true, true, true], X, [x[0], x[1], x[2], [4]], target);
    ApplyControlledOnBitString( [true, false, false, false], X, [x[0], x[1], x[2], [4]], target);
    ApplyControlledOnBitString( [true, false, true, false], X, [x[0], x[1], x[2], x[3]], target);
}

In [ ]:
open Microsoft.Quantum.Canon;

// This circuit implements a marking oracle that marks the letters that comprise the word COMPUTER.
// The oracle uses little endian to convert integers that represent letters positions in the aphabet to bit strings.
// The input will contain exactly 5 qubits.
operation QuantumCircuit(input : Qubit[], target : Qubit) : Unit is Adj {
    // If A is 0, then E will be 1
within{    X(input[0]); // Invert the control qubit to trigger on 0
    CNOT(input[0], input[4]);
    X(input[0]); // Revert the control qubit back to its original state

    // If B is 1 and C is 1, then E will be 1
    CCNOT(input[1], input[2], input[4]);

    // If C is 1 and D is 0, then E will be 1
    X(input[3]); // Invert the control qubit D to trigger on 0
    CCNOT(input[2], input[3], input[4]);
    X(input[3]); // Revert the control qubit D back to its original state

    // If A is 1, then E will be 1
    CNOT(input[0], input[4]);

    // If A is 1 and B is 0, then E will be 1
    X(input[1]); // Invert the control qubit B to trigger on 0
    CCNOT(input[0], input[1], input[4]);
    X(input[1]); // Revert the control qubit B back to its original state

    // If A is 1 and B is 1, then E will be 1
    CCNOT(input[0], input[1], input[4]);

    // If A is 0 and E is 0, then E will be 1
    X(input[0]); // Invert the control qubit A to trigger on 0
    CNOT(input[0], input[4]);
    X(input[0]); // Revert the control qubit A back to its original state

    // If A is 1 and D is 0, then E will be 1
    X(input[3]); // Invert the control qubit D to trigger on 0
    CCNOT(input[0], input[3], input[4]);
    X(input[3]); // Revert the control qubit D back to its original state
} apply {
    for q in input {
      X(q, target);
    }
        
    }
}

In [ ]:
%%qsharp
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;

// The operation that runs the oracle on every bit string and compares the results with those of a classical function
// (also checks that there are no measurements)
operation AssertOracleImplementsFunction (N : Int, oracle : ((Qubit[], Qubit) => Unit), f : (Bool[] -> Bool)) : Unit {
    let size = 1 <<< N;
    use (qs, target) = (Qubit[N], Qubit());
    for k in 0 .. size - 1 {
        // Prepare k-th bit vector
        let binary = IntAsBoolArray(k, N);
            
        //Message($"{k}/{N} = {binary}");
        // binary is little-endian notation, so the second vector tried has qubit 0 in state 1 and the rest in state 0
        ApplyPauliFromBitString(PauliX, true, binary, qs);
            
        // Apply the operation
        oracle(qs, target);
            
        // Check that the result is what we'd expect to measure
        let val = f(binary);
        if val {
            X(target);
        }
        if not CheckAllZero([target]) {
            fail $"Target in incorrect state for input {k} = {binary}";
        }

        // Check that the query qubits are still in the same state
        ApplyPauliFromBitString(PauliX, true, binary, qs);
        if not CheckAllZero(qs) {
            fail $"The state of control qubits changed for input {k} = {binary}";
        }
    }
}

// Check that the bitstring is one of the letters COMPUTER
function QuantumF(args : Bool[]) : Bool {
    let ascii = BoolArrayAsInt(args);
    return Any(letter -> letter == ascii, [3, 5, 13, 15, 16, 18, 20, 21]);
}

// Wrapper operation that allows to run the logical test for the task
operation TestWrapper() : Unit {
    let N = 5;
    AssertOracleImplementsFunction(N, QuantumCircuit, QuantumF);
    Message("Test passed!");
}

// Wrapper operation that allows to run resource estimation for the task.
// This operation only allocates the qubits and applies the oracle once, not using any additional gates or measurements.
operation ResourceEstimationWrapper() : Unit {
    let N = 5;
    use (input, target) = (Qubit[N], Qubit());
    QuantumCircuit(input, target);
}


Use the following cell to test your change to make sure it still performs the right computation. (You need to recompile the cell above every time you change your solution to test it.)
* If your solution is correct, the cell will report "Test passed!".
* Otherwise, it will throw an exception telling you on which basis state your solution is incorrect and in what way (either the function value is calculated incorrectly or the state of the control qubits changed).

In [ ]:
qsharp.eval("TestWrapper()")


Use the following cells to evaluate the resources required to run your solution.
* The score for the original code is 6048. 
* If your solution gets a score between 4500 and 6000, you'll get 1 point.
* If your solution gets a score between 2000 and 4500, you'll get 2 points.
* If your solution gets a score under 2000, you'll get 3 points.

In [ ]:
result = qsharp.estimate("ResourceEstimationWrapper()")
# result

In [ ]:
# The function that extracts the relevant resource information from the resource estimation job results and produces your absolute score.
def evaluate_results(res) : 
    width = res['physicalCounts']['breakdown']['algorithmicLogicalQubits']
    depth = res['physicalCounts']['breakdown']['algorithmicLogicalDepth']
    print(f"Logical algorithmic qubits = {width}")
    print(f"Algorithmic depth = {depth}")
    print(f"Score = {width * depth}")
    return width * depth


In [ ]:
evaluate_results(result)